In [1]:
import numpy as np
import xarray as xr
from tqdm import tqdm
from xcape.core import calc_cape
import os

In [2]:
idx=100
ds_t = xr.open_dataset(
        "/scratch/nf33/hk25_LSP/Germany/MC/ta_6hourly.nc").isel(time=idx)

ds_tdew = xr.open_dataset(
    "/scratch/nf33/hk25_LSP/Germany/MC/tdew_6hourly.nc").isel(time=idx)



In [3]:
## flip the pressure levels bcos so it goes from surface (descending order)
ta_da = ds_t["ta"].stack(ngrid=('latitude','longitude'))\
.isel(pressure=slice(None, None, -1)).transpose('ngrid','pressure')-273.15
tdew_da = ds_tdew["tdew"].stack(ngrid=('latitude','longitude'))\
.isel(pressure=slice(None, None, -1)).transpose('ngrid','pressure')-273.15

ta_surf = ds_t["ta"].isel(pressure=-1).stack(ngrid=('latitude','longitude')).values.flatten()-273.15
tdew_surf = ds_tdew["tdew"].isel(pressure=-1).stack(ngrid=('latitude','longitude')).values.flatten()-273.15
        


In [9]:
ta_val = ta_da.load().values
td_val = tdew_da.load().values
psurf = np.zeros(ta_val.shape[0])
psurf[:] = 1000
# ICON pressure is in Pa not hPa
plvl = ds_t.pressure.isel(pressure=slice(None, None, -1)).values.astype(float)/100        

cape, cin, mulev, zmulev = calc_cape(plvl, ta_val, td_val, 
                                         psurf,ta_surf, 
                                         tdew_surf,
                                         source ='most-unstable', vertical_lev="pressure")

run the py script to get CAPE and CIN for the entire dataset

In [11]:
ds_cape = xr.open_mfdataset("/scratch/nf33/hk25_LSP/Germany/MC/cape_cin/*.nc")

In [12]:
ds_cape.to_netcdf("/scratch/nf33/hk25_LSP/Germany/MC/cape_cin_6hourly.nc")

/jobfs/140911377.gadi-pbs/ipykernel_273473/3375606680.py:1: UserWarning: Times can't be serialized faithfully to int64 with requested units 'days since 2020-03-01'. Serializing with units 'hours since 2020-03-01' instead. Set encoding['dtype'] to floating point dtype to serialize with units 'days since 2020-03-01'. Set encoding['units'] to 'hours since 2020-03-01' to silence this warning .
  ds_cape.to_netcdf("/scratch/nf33/hk25_LSP/Germany/MC/cape_cin_6hourly.nc")
